# Creating a feature group 

A _feature store_ contains curated data assets that can be used as input for machine learning models, be it for training or inferencing. In IBM Cloud Pak for Data, a _catalog_ takes on the role of a feature store. The curated data assets are _feature groups_, which are data assets with additional metadata relevant for machine learning. 

This notebook demonstrates how to work with the `assetframe-lib` Python library to create feature groups. You can then publish these feature groups to a catalog for reuse.

## What you'll learn in this notebook

- How to create feature group metadata
- How to create a new data asset based on a pandas DataFrame, which includes feature group metadata
- How to add various attributes of feature group metadata, for example roles, value descriptions, bias information, or recipes. 

## Contents

This notebook contains the following parts:

1. [Create feature group metadata for a pandas DataFrame](#featuregroup)
2. [Create a new data asset with feature group information](#store)
3. [Summary](#summary)

<a id="featuregroup"></a>
## Create feature group metadata for a pandas DataFrame

Start by retrieving a CSV file with the `German Risk Credit` data set, and store this in a data frame. In a more typical use case, you'd create this data frame from one of your enterprise data sources.

In [1]:
import pandas as pd
credit_risk_df = pd.read_csv("https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/data/bias/german_credit_data_biased_training.csv")

In [2]:
credit_risk_df.head()

CheckingStatus  LoanDuration           CreditHistory LoanPurpose  \
0       0_to_200            31    credits_paid_to_date       other   
1         less_0            18    credits_paid_to_date     car_new   
2         less_0            15  prior_payments_delayed   furniture   
3       0_to_200            28    credits_paid_to_date  retraining   
4    no_checking            28  prior_payments_delayed   education   

   LoanAmount ExistingSavings EmploymentDuration  InstallmentPercent     Sex  \
0        1889      100_to_500             less_1                   3  female   
1         462        less_100             1_to_4                   2  female   
2         250        less_100             1_to_4                   2    male   
3        3693        less_100          greater_7                   3    male   
4        6235     500_to_1000          greater_7                   3    male   

  OthersOnLoan  ...       OwnsProperty Age  InstallmentPlans Housing  \
0         none  ...  savings_insurance  32              none     own   
1         none  ...  savings_insurance  37            stores     own   
2         none  ...        real_estate  28              none     own   
3         none  ...  savings_insurance  32              none     own   
4         none  ...            unknown  57              none     own   

  ExistingCreditsCount      Job Dependents  Telephone ForeignWorker     Risk  
0                    1  skilled          1       none           yes  No Risk  
1                    2  skilled          1       none           yes  No Risk  
2                    2  skilled          1        yes            no  No Risk  
3                    1  skilled          1       none           yes  No Risk  
4                    2  skilled          1       none           yes     Risk  

[5 rows x 21 columns]

### Create an AssetFrame object

To attach feature group metadata, you need an `AssetFrame` object. You initialize this object with the pandas `dataframe` that you eventually want to store as a feature group, and the `name` of the feature group.

In [3]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()
from assetframe_lib import AssetFrame

AssetFrame._wslib = wslib;

af = AssetFrame.from_pandas(dataframe=credit_risk_df, name="Credit Risk Training Data")

### Set the role of each feature

A `feature group` consists of individual `features`. Each feature has metadata that helps downstream machine learning tasks. This includes:
- The _role_ of the feature: Should it be used as _input_ to a machine learning model, is it the _target_ of a prediction, or is it the _identifier_ of a partocular row of a data asset.
- The _description_ and _tags_ that describe what a particular feature is about.
- The _fairness information_: this includes _monitored_ and _reference_ groups for input features, and the _favorable outcome_ for target features.
- A _recipe_ that describes how a feature was created. This is useful for features that are derived with a complex formula. 
- The _value descriptions_ that explain what particular values mean. This is useful for features that have values like `0`or `1`, but _mean_ something different (for example, they describe the marital status)

You can decide to include all columns of your DataFrame as features, or only a subset. We recommend that you filter down your DataFrame to the relevant columns first (using pandas), and then use _all_ these columns as features. Why? Because you benefit from the following shortcut:

In [4]:
allFeatures = af.create_default_features()

With this call, the AssetFrame will include all columns of the DataFrame as `feature` objects. It will also set the role of each feature to `Input`. You can see the result here:

In [5]:
af

Role Description Favorable labels  \
CheckingStatus            Input                                
LoanDuration              Input                                
CreditHistory             Input                                
LoanPurpose               Input                                
LoanAmount                Input                                
ExistingSavings           Input                                
EmploymentDuration        Input                                
InstallmentPercent        Input                                
Sex                       Input                                
OthersOnLoan              Input                                
CurrentResidenceDuration  Input                                
OwnsProperty              Input                                
Age                       Input                                
InstallmentPlans          Input                                
Housing                   Input                                
ExistingCreditsCount      Input                                
Job                       Input                                
Dependents                Input                                
Telephone                 Input                                
ForeignWorker             Input                                
Risk                      Input                                

                         Unfavorable labels Monitored groups Reference groups  \
CheckingStatus                                                                  
LoanDuration                                                                    
CreditHistory                                                                   
LoanPurpose                                                                     
LoanAmount                                                                      
ExistingSavings                                                                 
EmploymentDuration                                                              
InstallmentPercent                                                              
Sex                                                                             
OthersOnLoan                                                                    
CurrentResidenceDuration                                                        
OwnsProperty                                                                    
Age                                                                             
InstallmentPlans                                                                
Housing                                                                         
ExistingCreditsCount                                                            
Job                                                                             
Dependents                                                                      
Telephone                                                                       
ForeignWorker                                                                   
Risk                                                                            

                         Value descriptions Recipe Tags  
CheckingStatus                                           
LoanDuration                                             
CreditHistory                                            
LoanPurpose                                              
LoanAmount                                               
ExistingSavings                                          
EmploymentDuration                                       
InstallmentPercent                                       
Sex                                                      
OthersOnLoan                                             
CurrentResidenceDuration                                 
OwnsProperty                                             
Age                                                      
InstallmentPlans       

The default assignment works for features except for `Risk`, which is the target column in this training data set. So, you need to change its role by:
- Retrieving the feature `Risk` from the asset frame
- Setting the role of the feature to `Target`

In [6]:
risk_feat = af.get_feature("Risk")
risk_feat.set_roles(["Target"])

Printing the asset frame shows that the change was applied.

In [7]:
af

Role Description Favorable labels  \
CheckingStatus             Input                                
LoanDuration               Input                                
CreditHistory              Input                                
LoanPurpose                Input                                
LoanAmount                 Input                                
ExistingSavings            Input                                
EmploymentDuration         Input                                
InstallmentPercent         Input                                
Sex                        Input                                
OthersOnLoan               Input                                
CurrentResidenceDuration   Input                                
OwnsProperty               Input                                
Age                        Input                                
InstallmentPlans           Input                                
Housing                    Input                                
ExistingCreditsCount       Input                                
Job                        Input                                
Dependents                 Input                                
Telephone                  Input                                
ForeignWorker              Input                                
Risk                      Target                                

                         Unfavorable labels Monitored groups Reference groups  \
CheckingStatus                                                                  
LoanDuration                                                                    
CreditHistory                                                                   
LoanPurpose                                                                     
LoanAmount                                                                      
ExistingSavings                                                                 
EmploymentDuration                                                              
InstallmentPercent                                                              
Sex                                                                             
OthersOnLoan                                                                    
CurrentResidenceDuration                                                        
OwnsProperty                                                                    
Age                                                                             
InstallmentPlans                                                                
Housing                                                                         
ExistingCreditsCount                                                            
Job                                                                             
Dependents                                                                      
Telephone                                                                       
ForeignWorker                                                                   
Risk                                                                            

                         Value descriptions Recipe Tags  
CheckingStatus                                           
LoanDuration                                             
CreditHistory                                            
LoanPurpose                                              
LoanAmount                                               
ExistingSavings                                          
EmploymentDuration                                       
InstallmentPercent                                       
Sex                                                      
OthersOnLoan                                             
CurrentResidenceDuration                                 
OwnsProperty                                             
Age                                                      
In

### Set the fairness information

You want to ensure that users of the feature group know:
- The favorable and unfavorable outcome of `Risk`.
- That the columns `Sex`, `Age` and `LoanPurpose` are susceptible to bias. 
  - For `Sex`, you want to ensure that any model built is fair towards females. 
  - For `Age`, you want to ensure that any model built is fair to customers below 25. 
  - For `LoanPurpose`, you want to ensure that any model built is fair to customers who want to get a loan for education or retraining purposes.

For `Risk`, you will set the favorable and unfavorable values as follows:

In [8]:
risk_feat.set_favorable_labels("No Risk")
risk_feat.set_unfavorable_labels("Risk")

For `Sex`, you will set the _monitored_ group to `female` , and the reference group to `male`

In [9]:
sex_feat = af.get_feature("Sex")
sex_feat.set_reference_groups("male")
sex_feat.set_monitored_groups("female")

`Age` can take a range of integer values. Instead of specifying each age individually, you will specify a `range`. This range must be included between square brackets (`[]`).

In [10]:
age_feat = af.get_feature("Age")
age_feat.set_monitored_groups("[0,25]")
age_feat.set_reference_groups("[26,80]")

For `LoanPurpose`, you will use arrays to specify two values for monitored groups, and multiple values as reference groups.

In [11]:
credit_risk_df["LoanPurpose"].value_counts()

car_new       945
furniture     853
car_used      808
radio_tv      755
appliances    561
repairs       283
vacation      205
education     167
retraining    164
business      146
other         113
Name: LoanPurpose, dtype: int64

In [12]:
loan_feat = af.get_feature("LoanPurpose")
loan_feat.set_monitored_groups(['education', 'retraining'])
loan_feat.set_reference_groups(['car_new', 'furniture', 'car_used', 'radio_tv', 'appliances', 'repairs', 'vacation', 'business'])

`get_fairness_info()` returns the fairness information for all features as a Python `dict`. This can be used directly in various fairness evaluations, for example, in AutoAI or [AI Fairness 360](https://aif360.mybluemix.net/resources#overview).

In [13]:
af.get_fairness_info()

{'favorable_labels': ['No Risk'],
 'unfavorable_labels': ['Risk'],
 'protected_attributes': [{'feature': 'LoanPurpose',
   'monitored_group': ['education', 'retraining'],
   'reference_group': ['car_new',
    'furniture',
    'car_used',
    'radio_tv',
    'appliances',
    'repairs',
    'vacation',
    'business']},
  {'feature': 'Sex',
   'monitored_group': ['female'],
   'reference_group': ['male']},
  {'feature': 'Age',
   'monitored_group': [[0.0, 25]],
   'reference_group': [[26, 80]]}]}

You can also see the result by running the following command. `head()` displays the first rows of the data set, just like pandas, but also includes roles and fairness information. Column values that fall into monitored and reference groups are highlighted. Favorable and unfavorable labels are highlighted as well.

In [14]:
af.head()

### Set feature descriptions

If the orignal column name is not descriptive enough, you might want to set a feature description as shown in the following command.

In [15]:
credit_risk_df.columns

Index(['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose',
       'LoanAmount', 'ExistingSavings', 'EmploymentDuration',
       'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration',
       'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing',
       'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone',
       'ForeignWorker', 'Risk'],
      dtype='object')

In [16]:
install_feat = af.get_feature("InstallmentPercent")
install_feat.set_description("Installment rate in percentage of income")

dep_feat = af.get_feature("Dependents")
dep_feat.set_description("Number of people being liable to provide maintenance for")

You can tailor `head()`to also show these descriptions.

In [17]:
af.head(display_options=['role', 'description'])

### Set value descriptions
Although the feature name itself is clear, it might very well contain cryptic values. Setting a value description as shown below can help you find these values.

In [18]:
plan_feat = af.get_feature("InstallmentPlans")

val_descriptions = [
    {'value': 'stores', 'description': 'customer has additional business installment plan'},
    {'value': 'bank', 'description': 'customer has additional personal installment plan'},
    {'value': 'none', 'description': 'customer has no additional installment plan'}
]

plan_feat.set_value_descriptions(val_descriptions)

In [19]:
plan_feat

Column: InstallmentPlans
Roles:  Input
Value descriptions: ('stores', 'customer has additional business installment plan'), ('bank', 'customer has additional personal installment plan'), ('none', 'customer has no additional installment plan')

### Set the recipe

Some features might describe an established formula, or have been derived using specific code. By setting the `recipe`, you can remove any ambiguity that might exist because of how the feature was created from the raw data.

In [20]:
cred_hist_feat = af.get_feature("Telephone")

cred_hist_feat.set_recipe("(cust_phone_num) != null")

You can show the recipe in the data preview as well:

In [21]:
af.head(display_options=['role', 'description', 'recipe'])

Additionally, you can see all metadata directly on the asset frame:

In [22]:
af

Role  \
CheckingStatus             Input   
LoanDuration               Input   
CreditHistory              Input   
LoanPurpose                Input   
LoanAmount                 Input   
ExistingSavings            Input   
EmploymentDuration         Input   
InstallmentPercent         Input   
Sex                        Input   
OthersOnLoan               Input   
CurrentResidenceDuration   Input   
OwnsProperty               Input   
Age                        Input   
InstallmentPlans           Input   
Housing                    Input   
ExistingCreditsCount       Input   
Job                        Input   
Dependents                 Input   
Telephone                  Input   
ForeignWorker              Input   
Risk                      Target   

                                                                Description  \
CheckingStatus                                                                
LoanDuration                                                                  
CreditHistory                                                                 
LoanPurpose                                                                   
LoanAmount                                                                    
ExistingSavings                                                               
EmploymentDuration                                                            
InstallmentPercent                 Installment rate in percentage of income   
Sex                                                                           
OthersOnLoan                                                                  
CurrentResidenceDuration                                                      
OwnsProperty                                                                  
Age                                                                           
InstallmentPlans                                                              
Housing                                                                       
ExistingCreditsCount                                                          
Job                                                                           
Dependents                Number of people being liable to provide maint...   
Telephone                                                                     
ForeignWorker                                                                 
Risk                                                                          

                         Favorable labels Unfavorable labels  \
CheckingStatus                                                 
LoanDuration                                                   
CreditHistory                                                  
LoanPurpose                                                    
LoanAmount                                                     
ExistingSavings                                                
EmploymentDuration                                             
InstallmentPercent                                             
Sex                                                            
OthersOnLoan                                                   
CurrentResidenceDuration                                       
OwnsProperty                                                   
Age                                                            
InstallmentPlans                                               
Housing                                                        
ExistingCreditsCount                                           
Job                                                            
Dependents                                                     
Telephone                                                      
ForeignWorker                                                  
Risk                              No Risk               Risk   

                               Monitored groups  \
CheckingStatus                      

<a id="store"></a>
## Create a new data asset with the feature group information

Now you're ready to create the feature group. The following command will create a data asset in your project with the contents of your DataFrame _and_ the feature group metadata. It will use the `name` of the asset frame as the name of the data asset. Setting `overwrite_data=True` ensures that, if you re-run the notebook and change the pandas DataFrame, the data changes will be written to the asset. 

In [23]:
af.to_data_asset(overwrite_data=True)

{'steps': ['Create new data asset',
  'Create feature_group',
  'Create column_info'],
 'asset_name': 'Credit Risk Training Data',
 'asset_id': '3d333f3c-043d-4422-aa67-cbc80fd26920'}

<a id="summary"></a>
## Summary

Congratulations! You created a feature group with credit risk training data that provides valuable information for your fellow data scientists. Now, you can:
- Open the data asset in the project UI in Cloud Pak for Data, and use the Feature group tab to view and change the information further.
- Publish the data asset to a catalog. This will turn your catalog into a _feature store_ and users can start using your feature group across projects.

- See how easy it is to find feature groups. All you need to do is to:
  - Save and close this notebook.
  - Search for `Credit` in your workspace. (The search bar at the top of this screen)
  - Wait for the results to appear.
  - In the `Filters` section on the left, select `Type`: _Data_.
  - Now, scroll down the `Filters` section. You will see a _new_ filter: `Contains: Feature Group`entry. 
  - Select the checkbox and click `Apply`.
  - The results will be filtered down to data assets that are feature groups. This helps everybody find well-curated data for creating machine leraning models!

### Authors

**Alexander Lang**, Architect, Watson Studio, IBM